# Appmode in Jupyter

This page demonstrates how to generate online "apps" with a Jupyter Notebook interface. With the `appmode` plugin, we can create interactive experiences without requiring coding or running a specific cell.

Markdown cells will be retained, and all code cells will be run, then hidden. The outputs of each cell will be displayed as well.

Check out a further example using an [ipyvolume](https://ipyvolume.readthedocs.io/en/latest/) to render a 3D plot: [ipyvolume_demo.ipynb](ipyvolume_demo.ipynb)

In [ ]:
from __future__ import division

import pandas as pd

bulk_cmte_data_file = './campaign_finance_data/2019-2020_company_to_campaign_bulk_data.txt'
columns=['CMTE_ID', 'AMNDT_IND', 'RPT_TP', 'TRANSACTION_PGI', 'IMAGE_NUM', 'TRANSACTION_TP', 'ENTITY_TP', 'NAME', 'CITY', 'STATE', 'ZIP_CODE','EMPLOYER','OCCUPATION','TRANSACTION_DT','TRANSACTION_AMT','OTHER_ID','CAND_ID','TRAN_ID','FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID']
donations_df = pd.read_csv(bulk_cmte_data_file, sep="|", header=None, low_memory=False, names=columns)

cmte_id_key_file = './campaign_finance_data/2019-2020_company_master.txt'
cmte_id_cols=['CMTE_ID', 'CMTE_NM', 'TRES_NM', 'CMTE_ST1', 'CMTE_ST2', 'CMTE_CITY', 'CMTE_ST', 'CMTE_ZIP', 'CMTE_DSGN', 'CMTE_TP', 'CMTE_PTY_AFFILIATION','CMTE_FILING_FREQ','ORG_TP','CONNECTED_ORG_NM','CAND_ID']
cmte_df = pd.read_csv(cmte_id_key_file, sep="|", header=None, low_memory=False, names=cmte_id_cols, index_col=0)

cand_id_key_file = './campaign_finance_data/2019-2020_candidate_master.txt'
cand_id_cols=['CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_ELECTION_YR', 'CAND_OFFICE_ST', 'CAND_OFFICE', 'CAND_OFFICE_DISTRICT', 'CAND_ICI', 'CAND_STATUS', 'CAND_PCC', 'CAND_ST1','CAND_ST2','CAND_CITY','CAND_ST','CAND_ZIP']
cand_df = pd.read_csv(cand_id_key_file, sep="|", header=None, low_memory=False, names=cand_id_cols, index_col=0)

import ipywidgets as ipw

cb_input = ipw.Combobox(description='', options=[str(v) for v in cmte_df.CMTE_NM.tolist()], placeholder="Start typing here.")
html_output = ipw.HTML()

def on_go(btn):
    cmte_id_sel = cmte_df.index[cmte_df.CMTE_NM == cb_input.value]
    if cmte_id_sel.empty:
        # TODO make popup of company not found
        return
    sc_df = get_scorecard(cmte_id_sel.format()[0])
    html_output.value = sc_df.to_html(header=False,columns=['CAND_NAME','TRANSACTION_AMT'],index=False)
    #disp_df = sc_df.reindex(columns=['CAND_NAME','TRANSACTION_AMT'])

def on_clear(btn):
    cb_input.value = ''
    html_output.value = ''
    
def get_scorecard(cmte_id):
    cmte_donations_df = donations_df[donations_df.CMTE_ID == cmte_id] # Select all donations where the CMTE_ID == cmte_id_sel

    cmte_donations_df = cmte_donations_df.groupby('CAND_ID').TRANSACTION_AMT.sum()   # Sum transactions
    cmte_donations_df = pd.DataFrame(cmte_donations_df)
    cmte_donations_df = cmte_donations_df.merge(cand_df, how='left', left_index=True, right_index=True, suffixes=('',''))   # Merge candidate data into result
    
    return cmte_donations_df
    

go_button = ipw.Button(description='Go!', layout=ipw.Layout(width="45px"))
go_button.on_click(on_go)

clear_button = ipw.Button(description='Clear', layout=ipw.Layout(width="90px"))
clear_button.on_click(on_clear)

row1 = ipw.HBox((cb_input, go_button, clear_button))
row2 = html_output
ipw.VBox((row1, row2))

